In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-01-2020


In [5]:
state = input('state, e.g. New Jersey')

state, e.g. New Jersey New Hampshire


### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.223334,-82.461707,4,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295065,-92.414197,47,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-01 21:58:49,37.767072,-75.632346,7,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-01 21:58:49,43.452658,-116.241552,195,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-01 21:58:49,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2229.000000,2229.000000,2230.000000,2230.000000,2230.000000,2230.0
mean,NaN,37.255595,-88.693085,95.682511,2.133184,3.800000,0.0
std,NaN,7.067256,18.464507,1096.510006,27.897826,179.446928,0.0
min,NaN,0.000000,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,33.964725,-95.563511,2.000000,0.000000,0.000000,0.0
50%,NaN,37.871198,-87.787962,5.000000,0.000000,0.000000,0.0
75%,NaN,41.435647,-81.962311,23.000000,1.000000,0.000000,0.0
max,NaN,64.807262,145.673900,47439.000000,1139.000000,8474.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.head()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,0.0,1950.187952,-5119.386303,1060,27,0,0
Alaska,0.0,362.539360,-862.001024,132,3,0,0
Arizona,0.0,505.103399,-1671.360527,1530,29,0,0
Arkansas,0.0,1846.341710,-4900.768064,584,10,0,0
California,0.0,1873.911674,-6028.510306,9399,199,0,0


In [10]:
state_totals['death_rate?'] = state_totals['Deaths'] / state_totals['Confirmed'] * 100.0

In [11]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active,death_rate?
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,57.000000
mean,0.0,1431.771054,-3408.567003,3678.827586,82.017241,146.103448,0.0,2.308301
std,0.0,1180.539394,2951.908853,11324.251290,261.141589,1112.690712,0.0,2.219763
min,0.0,0.000000,-13764.875282,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,483.170922,-5253.566369,290.250000,5.000000,0.000000,0.0,1.535836
50%,0.0,1189.846001,-3076.060812,728.500000,17.500000,0.000000,0.0,2.020057
75%,0.0,2368.468833,-864.637952,2840.750000,65.000000,0.000000,0.0,2.552022
max,0.0,4566.435219,145.673900,83948.000000,1941.000000,8474.000000,0.0,16.666667


In [12]:
state_covid = df[df.Province_State == state].copy()
state_covid['death_rate'] = 100 * state_covid['Deaths'] / state_covid['Confirmed']
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'death_rate' ]

In [13]:
sort_by = input('sort by one of Admin2, Confirmed, Deaths, death_rate')

sort by one of Admin2, Confirmed, Deaths, death_rate Confirmed


In [14]:
state_covid[state_columns].sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate
FIPS,,,,,,
33015.0,Rockingham,137,1,0,0,0.729927
33011.0,Hillsborough,99,2,0,0,2.020202
33009.0,Grafton,34,0,0,0,0.000000
33017.0,Strafford,28,0,0,0,0.000000
33013.0,Merrimack,27,0,0,0,0.000000
33001.0,Belknap,12,0,0,0,0.000000
33003.0,Carroll,12,0,0,0,0.000000
NaN,Manchester,5,0,0,0,0.000000
NaN,Nashua,5,0,0,0,0.000000


## We need to locate county data to add per capita rates.

In [15]:
data = Path('../data/counties')
state_pop_path = data / Path(state.replace(' ', '').lower() + '.csv')

In [16]:
state_pop_path

PosixPath('../data/counties/newhampshire.csv')

In [17]:
state_pop = pd.read_csv(state_pop_path)

In [18]:
state_with_population = pd.merge(state_covid[state_columns], state_pop,
                                 left_on='Admin2', right_on='county')

state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 100.0
state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 100.0
state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 100.0

sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")

state_with_population.sort_values(by=sort_by, ascending=False)

column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate? fraction_confirmed


,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
6,Rockingham,137,1,0,0,0.729927,2,Rockingham,305129,0.044899,0.000328,0.729927
3,Grafton,34,0,0,0,0.000000,5,Grafton,89811,0.037857,0.000000,0.000000
1,Carroll,12,0,0,0,0.000000,8,Carroll,47840,0.025084,0.000000,0.000000
4,Hillsborough,99,2,0,0,2.020202,1,Hillsborough,411087,0.024082,0.000487,2.020202
7,Strafford,28,0,0,0,0.000000,4,Strafford,128237,0.021835,0.000000,0.000000
0,Belknap,12,0,0,0,0.000000,7,Belknap,60640,0.019789,0.000000,0.000000
5,Merrimack,27,0,0,0,0.000000,3,Merrimack,149452,0.018066,0.000000,0.000000
8,Sullivan,4,0,0,0,0.000000,9,Sullivan,43125,0.009275,0.000000,0.000000
2,Cheshire,4,0,0,0,0.000000,6,Cheshire,76263,0.005245,0.000000,0.000000


In [20]:
state_with_population.sort_values(by='Deaths', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
4,Hillsborough,99,2,0,0,2.020202,1,Hillsborough,411087,0.024082,0.000487,2.020202
6,Rockingham,137,1,0,0,0.729927,2,Rockingham,305129,0.044899,0.000328,0.729927
0,Belknap,12,0,0,0,0.000000,7,Belknap,60640,0.019789,0.000000,0.000000
1,Carroll,12,0,0,0,0.000000,8,Carroll,47840,0.025084,0.000000,0.000000
2,Cheshire,4,0,0,0,0.000000,6,Cheshire,76263,0.005245,0.000000,0.000000
3,Grafton,34,0,0,0,0.000000,5,Grafton,89811,0.037857,0.000000,0.000000
5,Merrimack,27,0,0,0,0.000000,3,Merrimack,149452,0.018066,0.000000,0.000000
7,Strafford,28,0,0,0,0.000000,4,Strafford,128237,0.021835,0.000000,0.000000
8,Sullivan,4,0,0,0,0.000000,9,Sullivan,43125,0.009275,0.000000,0.000000


In [21]:
state_with_population.sort_values(by='death_rate', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
4,Hillsborough,99,2,0,0,2.020202,1,Hillsborough,411087,0.024082,0.000487,2.020202
6,Rockingham,137,1,0,0,0.729927,2,Rockingham,305129,0.044899,0.000328,0.729927
0,Belknap,12,0,0,0,0.000000,7,Belknap,60640,0.019789,0.000000,0.000000
1,Carroll,12,0,0,0,0.000000,8,Carroll,47840,0.025084,0.000000,0.000000
2,Cheshire,4,0,0,0,0.000000,6,Cheshire,76263,0.005245,0.000000,0.000000
3,Grafton,34,0,0,0,0.000000,5,Grafton,89811,0.037857,0.000000,0.000000
5,Merrimack,27,0,0,0,0.000000,3,Merrimack,149452,0.018066,0.000000,0.000000
7,Strafford,28,0,0,0,0.000000,4,Strafford,128237,0.021835,0.000000,0.000000
8,Sullivan,4,0,0,0,0.000000,9,Sullivan,43125,0.009275,0.000000,0.000000
